# Build a chatbot with history

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AD1A947320>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AD1A947E60>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi I am guru and I am a data scientist")])

AIMessage(content="Hi Guru! It's nice to meet you.  \n\nThat's awesome that you're a data scientist. What kind of work do you specialize in? \n\nI'm always interested in learning more about how people use data to solve problems and make insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 19, 'total_tokens': 78, 'completion_time': 0.107272727, 'prompt_time': 8.034e-05, 'queue_time': 0.020264169999999998, 'total_time': 0.107353067}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d51881fe-de9d-4d15-9dfd-46fc5569ea21-0', usage_metadata={'input_tokens': 19, 'output_tokens': 59, 'total_tokens': 78})

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi I am guru and I am a data scientist"),
        AIMessage(content="Hi Guru! It's nice to meet you.  \n\nThat's awesome that you're a data scientist. What kind of work do you specialize in? \n\nI'm always interested in learning more about how people use data to solve problems and make insights."),
        HumanMessage(content="Hey Whhat's my name what do I do?")      
    ]
)

AIMessage(content="You told me your name is Guru, and you're a data scientist! 😊  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.0034427, 'queue_time': 0.020834526, 'total_time': 0.087079064}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c8e680da-42e3-48a4-a9a0-dc5715e168a4-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

In [ ]:
!pip install langchain_community

In [14]:
## Message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

## This function ensures that every chat or session is different from the other
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [13]:
config={"configurable":{"session_id":"chat1"}}

In [16]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hey my name is Guru and I am data scientist")
    ],
    config=config
)

In [17]:
response.content

"Hi Guru, it's great to meet you! \n\nBeing a data scientist is a fascinating field. What kind of work do you specialize in?  Do you work with a specific industry or type of data?  I'm always eager to learn more about the exciting things people are doing with data! 😊  \n\n"

In [18]:
with_message_history.invoke(
    [
        HumanMessage(content="Hey What's my name?")
    ],
    config=config
)

AIMessage(content='Your name is Guru!  😊  \n\nI remember from our previous conversation. \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 179, 'total_tokens': 200, 'completion_time': 0.038181818, 'prompt_time': 0.005295265, 'queue_time': 0.019879584999999998, 'total_time': 0.043477083}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d61f6bed-1e2a-4e39-9dda-c2f0d10f9a80-0', usage_metadata={'input_tokens': 179, 'output_tokens': 21, 'total_tokens': 200})

In [22]:
#Chage the session id

config1={"configurable":{"session_id":"chat2"}}
response1=with_message_history.invoke(
    [
        HumanMessage(content="Hey my name is john")
    ],
    config=config1
)
response1.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today? 😄\n"

In [23]:
config1={"configurable":{"session_id":"chat2"}}
response1=with_message_history.invoke(
    [
        HumanMessage(content="What's my name")
    ],
    config=config1
)
response1.content

"Your name is John, remember? 😊 \n\nI'm here to help if you need anything else!  \n"

# Prompt templates

Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

Prompt Templates take as input a dictionary, where each key represents a variable in the prompt template to fill in.

Prompt Templates output a PromptValue. This PromptValue can be passed to an LLM or a ChatModel, and can also be cast to a string or a list of messages. The reason this PromptValue exists is to make it easy to switch between strings and messages.

In [25]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a useful assistant, Answer all the question with best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Guru")]})

AIMessage(content="Hello Guru, it's nice to meet you!  \n\nI'm glad you're here. What can I do for you today?  \n\nAsk me anything, and I'll do my best to help. 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 29, 'total_tokens': 81, 'completion_time': 0.094545455, 'prompt_time': 0.00014056, 'queue_time': 0.020152266999999998, 'total_time': 0.094686015}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7893c99-59cd-4ea3-b060-8a1f5a053b0f-0', usage_metadata={'input_tokens': 29, 'output_tokens': 52, 'total_tokens': 81})

In [27]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [28]:
config2={"configurable":{"session_id":"chat3"}}
response1=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Guru")],
    config=config2
)
response1.content

"Hello Guru! It's nice to meet you. \n\nI'm happy to help with any questions you might have. Just ask away! 😊  \n\n"

In [37]:
# Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a useful assistant, Answer all the question with best of your ability in {language}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [38]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Guru")],"language":"Hindi"})

response.content

'नमस्ते गुरु जी!  \n\nआज मैं आपकी मदद करने के लिए तैयार हूँ। कोई भी प्रश्न पूछें, मैं अपना सर्वश्रेष्ठ प्रयास करूँगा। 😊\n'

Let's wrap this more complicated chain in messages history

In [39]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [40]:
config3={"configurable":{"session_id":"chat4"}}

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am Guru")], "language":"Hindi"},
    config=config3
)

response.content

'नमस्ते गुरु जी! \n\nमुझे खुशी है कि आप मुझसे बात कर रहे हैं। आप मुझसे जो भी प्रश्न पूछेंगे, मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे बेहतर उत्तर दे सकूँ। \n\nआप क्या जानना चाहते हैं? 🤔 \n\n'

In [42]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")], "language":"Hindi"},
    config=config3
)

response.content

'आपने ही खुद अपनी पहचान बताई गुरु जी, आपका नाम "गुरु" है।  😊 \n\n\n'

## Menage the conversation history

'trim_message' = trim_messages can be used to reduce the size of a chat history to a specified token count or specified message count.

In [47]:
!pip install transformers

  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached safetensors-0.4.5-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 1.9 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/10.1 MB 1.5 MB/s eta 0:00:07
   ----- ---------------------------------- 1.3/10.1 MB 1.9 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/10.1 MB 2.2 MB/s eta 0:00:04
   ---------- ----------------------------- 2.6/10.1 MB 2.4 MB/s eta 0:00:04
   ------------ --------------------------- 3.1/10.1 MB 2

In [54]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi ! my name is lokas!"),
    AIMessage(content="Hi !"),
    HumanMessage(content="I like to play cricket"),
    AIMessage(content="Nice"),
    HumanMessage(content="Whats 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem !"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like to play cricket', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem !', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [56]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What game I like to play?")],
    "language":"English"
    }
)

response.content

"That's a tricky one! I don't actually know what games *you* like.\n\nTo figure that out, tell me: \n\n* What kind of things do you enjoy doing? \n* Do you prefer playing alone, with friends, or online?\n* Are you looking for a game that's relaxing, challenging, creative, or something else?\n\nThe more you tell me, the better I can help you find a game you'll love! 😊  \n\n"

In [59]:
#lets wraop this all in message history
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config4 = {"configurable":{"session_id":"chat5"}}


In [60]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")], "language":"English"},
    config=config4
)

response.content

"As an AI, I have no memory of past conversations and do not know your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [61]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What math's problem did I solved?")], "language":"English"},
    config=config4
)

response.content

'Please tell me the math problem you solved so I can tell you if I can help! 😊  \n\nI need to know the problem to confirm if you solved it correctly.  \n\n'